# Geospatial Python
## Accessing satellite imagery using Python
Setup: https://carpentries-incubator.github.io/geospatial-python/index.html

Instruction: https://carpentries-incubator.github.io/geospatial-python/05-access-data.html

Objectives:
* Search public [SpatioTemporal Asset Catalog (STAC)](https://github.com/radiantearth/stac-api-spec/tree/release/v1.0.0) repositories of satellite imagery using Python.
* Inspect search result’s metadata.
* Download (a subset of) the assets available for a satellite scene.
* Open satellite imagery as raster data and save it to disk.

In [41]:
# first import necessary libraries
import rioxarray # to open and download remote raster data
import pystac_client # to query STAC API endpoint
import planetary_computer
from shapely.geometry import Point # to create a point 
import geojson # to parse spatial data format
import folium # to create an interactive map
from folium.plugins import Draw # to allow drawing
from localtileserver import TileClient, get_folium_tile_layer # to visualize the geotif 

In [8]:
center_coord = [40.60104027382292, -105.09137099497742] # fort collins
m  = folium.Map(center_coord, zoom_start=5)
draw = Draw(export=True)
draw.add_to(m)
m

In [29]:
api_url = "https://earth-search.aws.element84.com/v1"

# open the api
client = Client.open(api_url)

In [30]:
# perform metadata search limited to 10 results from Sentinel-2, Level 2A, to retrieve Cloud Optimized GeoTiffs (COGs)

# store a variable pointing to the collection of interest
# Note: collection ID is taken from Sentinel-2 Level 2A - https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-c1-l2a
collection = "sentinel-2-l2a" 
'''
includes Sentinel-2 data products 
pre-processed at level 2A (bottom-of-atmosphere reflectance) 
and saved in Cloud Optimized GeoTIFF (COG) format:
'''

# create a point to intersect from, note values are in format x (long), y (lat) https://shapely.readthedocs.io/en/stable/reference/shapely.Point.html
point = Point(-105.275116,40.605531,)  # Use interactive map above 
# or use https://www.google.com/maps > search for place > right click to access lat and lng
# lat (y) usually displayed before lng x, lng goes from -180 (west of Greenwich) - 0 - 180
search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
)
# show the number of scenes (i.e. the portion of the footage recorded by the satellite)
print(search.matched())

1295


In [31]:
# store the metadata of the search results
items = search.item_collection()

In [32]:
# get the length of items
print(len(items))

10


In [33]:
# loop over the items to get there ids
for item in items:
    print(item)

<Item id=S2B_13TDE_20240629_0_L2A>
<Item id=S2B_13TDF_20240629_0_L2A>
<Item id=S2A_13TDE_20240624_0_L2A>
<Item id=S2A_13TDF_20240624_0_L2A>
<Item id=S2B_13TDE_20240619_0_L2A>
<Item id=S2B_13TDF_20240619_0_L2A>
<Item id=S2A_13TDE_20240614_0_L2A>
<Item id=S2A_13TDF_20240614_0_L2A>
<Item id=S2B_13TDE_20240609_0_L2A>
<Item id=S2B_13TDF_20240609_0_L2A>


In [34]:
#  inspect the metadata associated with the first item of the search result
item = items[0]
print(item.datetime)
print(item.geometry)
print(item.properties)

2024-06-29 18:02:59.350000+00:00
{'type': 'Polygon', 'coordinates': [[[-106.18316062292897, 40.64478591807637], [-106.1661201925584, 39.65576169772817], [-104.88623172845638, 39.661560248682505], [-104.88456877606482, 40.650789813976196], [-106.18316062292897, 40.64478591807637]]]}
{'created': '2024-06-30T00:17:03.571Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 20.563914, 'proj:epsg': 32613, 'mgrs:utm_zone': 13, 'mgrs:latitude_band': 'T', 'mgrs:grid_square': 'DE', 'grid:code': 'MGRS-13TDE', 'view:sun_azimuth': 137.368269070008, 'view:sun_elevation': 68.46217570265969, 's2:degraded_msi_data_percentage': 0.0366, 's2:nodata_pixel_percentage': 0, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 1.604882, 's2:cloud_shadow_percentage': 2.201446, 's2:vegetation_percentage': 38.534451, 's2:not_vegetated_percentage': 32.523867, 's2:water_percentage': 0.834702, 's2:unclassified_percentage': 3.273052, 's2:mediu

In [36]:
'''
EXERCISE: Search the sentinel-2-l2a collection for all the available scenes that satisfy the following criteria: 
- intersect a provided bounding box (use ±0.01 deg in lat/lon from the previously defined point); 
- have been recorded between 20 March 2020 and 30 March 2020; 
- have a cloud coverage less than 15. Note: the eo extension (https://github.com/stac-extensions/eo) is implemented in some collections allowing it to be queried against

* get the count
* save the results to json
'''
bbox = point.buffer(0.01).bounds

search = client.search(
    collections=[collection],
    bbox=bbox,
    datetime="2023-03-20/2024-03-30",
    query=["eo:cloud_cover<15"]
)
print(search.matched())
items = search.item_collection()
items.save_object("search.json") # json file saved alongside notebook

47


## Access the assets


In [37]:
# first item's assets
assets = items[0].assets  

print(assets.keys())

dict_keys(['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'thumbnail', 'tileinfo_metadata', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2', 'red-jp2', 'rededge1-jp2', 'rededge2-jp2', 'rededge3-jp2', 'scl-jp2', 'swir16-jp2', 'swir22-jp2', 'visual-jp2', 'wvp-jp2'])


In [38]:
# print a minimal description of the available assets
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

aot: Aerosol optical thickness (AOT)
blue: Blue (band 2) - 10m
coastal: Coastal aerosol (band 1) - 60m
granule_metadata: None
green: Green (band 3) - 10m
nir: NIR 1 (band 8) - 10m
nir08: NIR 2 (band 8A) - 20m
nir09: NIR 3 (band 9) - 60m
red: Red (band 4) - 10m
rededge1: Red edge 1 (band 5) - 20m
rededge2: Red edge 2 (band 6) - 20m
rededge3: Red edge 3 (band 7) - 20m
scl: Scene classification map (SCL)
swir16: SWIR 1 (band 11) - 20m
swir22: SWIR 2 (band 12) - 20m
thumbnail: Thumbnail image
tileinfo_metadata: None
visual: True color image
wvp: Water vapour (WVP)
aot-jp2: Aerosol optical thickness (AOT)
blue-jp2: Blue (band 2) - 10m
coastal-jp2: Coastal aerosol (band 1) - 60m
green-jp2: Green (band 3) - 10m
nir-jp2: NIR 1 (band 8) - 10m
nir08-jp2: NIR 2 (band 8A) - 20m
nir09-jp2: NIR 3 (band 9) - 60m
red-jp2: Red (band 4) - 10m
rededge1-jp2: Red edge 1 (band 5) - 20m
rededge2-jp2: Red edge 2 (band 6) - 20m
rededge3-jp2: Red edge 3 (band 7) - 20m
scl-jp2: Scene classification map (SCL)
swi

In [39]:
# show one metadata value
print(assets["thumbnail"])
print(assets["thumbnail"].href)

<Asset href=https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/13/T/DE/2024/3/S2B_13TDE_20240321_0_L2A/thumbnail.jpg>
https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/13/T/DE/2024/3/S2B_13TDE_20240321_0_L2A/thumbnail.jpg


In [40]:
# open nir with the rioxarray library
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
print(nir)

<xarray.DataArray (band: 1, y: 10980, x: 10980)> Size: 241MB
[120560400 values with dtype=uint16]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 88kB 4e+05 4e+05 4e+05 ... 5.097e+05 5.098e+05
  * y            (y) float64 88kB 4.5e+06 4.5e+06 4.5e+06 ... 4.39e+06 4.39e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0


In [37]:
# save whole image to disk - this may take awhile
#nir.rio.to_raster("nir.tif")

In [45]:
nir_tiles = TileClient(nir_href) # create tiles client
nir_layer = get_folium_tile_layer(nir_tiles, name='nir') # create elevation tile layer
m = folium.Map(location=[point.y,point.x], zoom_start=12)
nir_layer.add_to(m)

draw = Draw(export=True)
draw.add_to(m)

folium.LayerControl().add_to(m)
m

In [49]:
# save portion of an image to disk
geom='''{"type":"Polygon","coordinates":[[[-105.7901,39.91711],[-105.7901,40.210343],[-105.382233,40.210343],[-105.382233,39.91711],[-105.7901,39.91711]]]}'''
geojson.loads(geom)
cropping_geometries = [geojson.loads(geom)] # converts to list

# https://corteva.github.io/rioxarray/html/rioxarray.html#rioxarray-rio-accessors
nir_sub = nir.rio.clip(geometries=cropping_geometries, crs=4326)
nir_sub.rio.to_raster("nir_subset.tif")

In [54]:
nir_tiles = TileClient("nir_subset.tif") # create tiles client
nir_layer = get_folium_tile_layer(nir_tiles, name='nir') # create elevation tile layer
m = folium.Map(location=[point.y,point.x], zoom_start=12)
nir_layer.add_to(m)

draw = Draw(export=True)
draw.add_to(m)

folium.LayerControl().add_to(m)
m

/Users/kevin/anaconda3/envs/rem_environment/lib/python3.10/site-packages/rio_tiler/io/rasterio.py:130: NoOverviewWarning: The dataset has no Overviews. rio-tiler performances might be impacted.
  warnings.warn(
